# Uber Data Eng.:Transform and Model (LM)

In [44]:
import pandas as pd
import requests 
import io


In [41]:
url = 'https://storage.googleapis.com/uber_data_eng_bucket/uber_data.csv'
# getting the response object from our flat file
response = requests.get(url)

In [45]:
df = pd.read_csv(io.StringIO(response.text), sep=',')
df.shape

(100000, 19)

In [50]:
# Displaying flat hierarchy table
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80


## Transformation

In [22]:
# We first drop all the NaNs in the entire data set
df = df.drop_duplicates().reset_index(drop = True)

# Then a primary key is assigned
df['trip_id'] = df.index

In [48]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80


In [24]:
# We take a glance at the type of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   VendorID               100000 non-null  int64         
 1   tpep_pickup_datetime   100000 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  100000 non-null  datetime64[ns]
 3   passenger_count        100000 non-null  int64         
 4   trip_distance          100000 non-null  float64       
 5   pickup_longitude       100000 non-null  float64       
 6   pickup_latitude        100000 non-null  float64       
 7   RatecodeID             100000 non-null  int64         
 8   store_and_fwd_flag     100000 non-null  object        
 9   dropoff_longitude      100000 non-null  float64       
 10  dropoff_latitude       100000 non-null  float64       
 11  payment_type           100000 non-null  int64         
 12  fare_amount            100000 non-null  float

We see that `tpep_pickup_datetime` and `tpep_dropoff_datetime` are in object format, 
therefore we need to convert these (below) with the function to_datetime() 

In [18]:
# we see that tpep_pick uop and tep_dropoff are in object format, 
# we need to convert these (below) with the function to_datetime() 

In [25]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# We only want to chech these 2 columns, therefore:
df[['tpep_pickup_datetime', 
    'tpep_dropoff_datetime']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   tpep_pickup_datetime   100000 non-null  datetime64[ns]
 1   tpep_dropoff_datetime  100000 non-null  datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 1.5 MB


## Modeling : 

 Main explanation to be located within directory main readme.md

In [52]:
df.head(6)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80
5,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,5.92,-74.017197,40.705383,1,N,-73.978073,40.755787,1,23.5,1.0,0.5,5.06,0.00,0.3,30.36


We will first mainly create the series of tables which will be deployed further on within Mage, these are:

#### 'passenger_count_dim' table

In [39]:
# We generate 'passenger_count_dim' dataframe and the reset index
passenger_count_dim = df[['passenger_count']].reset_index(drop = True)

# Then we create an index column within it
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index

# And so we finally insert this new columns at the begining of the dataframe
in_col_passeng = passenger_count_dim.pop('passenger_count_id')

passenger_count_dim.insert(loc = 0,
                           value = in_col_passeng,
                           column = 'passenger_count_id')

passenger_count_dim.head()

,passenger_count_id,passenger_count
0,0,1
1,1,1
2,2,2
3,3,3
4,4,5


We will utilize this same pattern for the rest of the dimensional tables as follows below:

#### 'trip_distance_dim' table



In [29]:
trip_distance_dim = df[['trip_distance']].reset_index(drop = True)

trip_distance_dim['trip_distance_id'] = trip_distance_dim.index

in_col_dist = trip_distance_dim.pop('trip_distance_id')

trip_distance_dim.insert(loc = 0, 
                         value = in_col_dist,
                         column = 'trip_distance_id')

trip_distance_dim.head()

,trip_distance_id,trip_distance
0,0,2.50
1,1,2.90
2,2,19.98
3,3,10.78
4,4,30.43


#### 'pickup_location_dim' table



In [30]:
pickup_location_dim = df[['pickup_longitude',
                          'pickup_latitude']].reset_index(drop = True)

pickup_location_dim['pickup_location_id'] = pickup_location_dim.index

in_col_pick = pickup_location_dim.pop('pickup_location_id')

pickup_location_dim.insert(loc = 0,  
                           value = in_col_pick,
                           column = 'pickup_location_id')

pickup_location_dim.head()

,pickup_location_id,pickup_longitude,pickup_latitude
0,0,-73.976746,40.765152
1,1,-73.983482,40.767925
2,2,-73.782021,40.644810
3,3,-73.863419,40.769814
4,4,-73.971741,40.792183


#### 'dropoff_location_dim' table



In [36]:
dropoff_location_dim = df[['dropoff_longitude',
                           'dropoff_latitude']].reset_index(drop = True)

dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index

in_col_drop = dropoff_location_dim.pop('dropoff_location_id')

dropoff_location_dim.insert(loc = 0,  
                           value = in_col_drop,
                           column = 'dropoff_location_id')

dropoff_location_dim.head()

,dropoff_location_id,dropoff_longitude,dropoff_latitude
0,0,-74.004265,40.746128
1,1,-74.005943,40.733166
2,2,-73.974541,40.675770
3,3,-73.969650,40.757767
4,4,-74.177170,40.695053


#### 'datetime_dim' table



In [31]:
# We create new columns for datetime observations

# picking_up
datetime_dim = df[['tpep_pickup_datetime',
                   'tpep_dropoff_datetime']].reset_index(drop = True)

# We now create new columns for all these individual datetime observations, this is:
datetime_dim['pickup_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour  # fetches hours
datetime_dim['pickup_day'] = datetime_dim['tpep_pickup_datetime'].dt.day    # fetches day
datetime_dim['pickup_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday # etc..
datetime_dim['pickup_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pickup_year'] = datetime_dim['tpep_pickup_datetime'].dt.year

# dropping_off
datetime_dim['dropoff_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['dropoff_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['dropoff_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday
datetime_dim['dropoff_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['dropoff_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year

# Then we generate a column to have the position of each observation with the method index() 
datetime_dim['datetime_id'] = datetime_dim.index


# Shift 'datetime_id' column to first position
in_col_datetime = datetime_dim.pop('datetime_id')

datetime_dim.insert( loc = 0,
                     value = in_col_datetime,
                     column = 'datetime_id')

datetime_dim.head()

,datetime_id,tpep_pickup_datetime,tpep_dropoff_datetime,pickup_hour,pickup_day,pickup_weekday,pickup_month,pickup_year,dropoff_hour,dropoff_day,dropoff_weekday,dropoff_month,dropoff_year
0,0,2016-03-01,2016-03-01 00:07:55,0,1,1,3,2016,0,1,1,3,2016
1,1,2016-03-01,2016-03-01 00:11:06,0,1,1,3,2016,0,1,1,3,2016
2,2,2016-03-01,2016-03-01 00:31:06,0,1,1,3,2016,0,1,1,3,2016
3,3,2016-03-01,2016-03-01 00:00:00,0,1,1,3,2016,0,1,1,3,2016
4,4,2016-03-01,2016-03-01 00:00:00,0,1,1,3,2016,0,1,1,3,2016


#### 'rate_code_dim' table



In [42]:
# we create a dictionary as per the data catalogue 
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

#We generate a new dataframe and reset its indexes
rate_code_dim = df[['RatecodeID']].reset_index(drop = True)

# We create a new column 'rate_code_id' and assign the indexes to it
rate_code_dim['rate_code_id'] = rate_code_dim.index

# and we will map this new column 'rate_code_name' 
# according to the dictionary 'rate_code_type' created above 
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)

# We finally rearrange this new dataframe as follows
rate_code_dim = rate_code_dim[['rate_code_id'
                               ,'RatecodeID'
                               ,'rate_code_name']]

rate_code_dim.head()

,rate_code_id,RatecodeID,rate_code_name
0,0,1,Standard rate
1,1,1,Standard rate
2,2,1,Standard rate
3,3,1,Standard rate
4,4,3,Newark


#### 'payment_type_dim' table



In [43]:
# we will lastly utilize the same structure as preceeds above
# to create the last dimensional table 'payment_type_dim' 
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}

payment_type_dim = df[['payment_type']].reset_index(drop = True)

payment_type_dim['payment_type_id'] = payment_type_dim.index

payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)

payment_type_dim = payment_type_dim[['payment_type_id',
                                     'payment_type',
                                     'payment_type_name']]

payment_type_dim.head()

,payment_type_id,payment_type,payment_type_name
0,0,1,Credit card
1,1,1,Credit card
2,2,1,Credit card
3,3,1,Credit card
4,4,1,Credit card


We finally will elaborate the fact table by merging these dimnesional we've just created. This is to code:

In [38]:
fact_table = df.merge(passenger_count_dim,
                      left_on='trip_id',
                      right_on='passenger_count_id') \
             .merge(trip_distance_dim,
                    left_on='trip_id',
                    right_on='trip_distance_id') \
             .merge(pickup_location_dim,
                    left_on='trip_id',
                    right_on='pickup_location_id') \
             .merge(dropoff_location_dim,
                    left_on='trip_id',
                    right_on='dropoff_location_id')\
             .merge(datetime_dim,
                    left_on='trip_id',
                    right_on='datetime_id') \
             .merge(rate_code_dim,
                    left_on='trip_id',
                    right_on='rate_code_id') \
             .merge(payment_type_dim,
                    left_on='trip_id',
                    right_on='payment_type_id') \
                [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
                'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
                'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
                'improvement_surcharge', 'total_amount']]

fact_table.head()

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,1,1,1,1,1,N,1,1,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2,2,2,2,2,N,2,2,2,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,3,2,3,3,3,3,N,3,3,3,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,4,2,4,4,4,4,N,4,4,4,98.0,0.0,0.0,0.00,15.50,0.3,113.80


And so, the modeling and transformation work through our local machine is done now. Next, we will be uploading data into the colud storage so that we be able to deploy our code into Mage and conduct the ETL orchestration from within it.